In [ ]:
pip install bitmex

In [ ]:
import numpy as np
import pandas as pd
import math
import time
from bitmex import bitmex
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

In [ ]:
### API
bitmex_api_key = 'API_KEY'    #Enter your own API-key here
bitmex_api_secret = 'API_SECRET' #Enter your own API-secret here

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)

### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  
      old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "bitmex": 
      old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "bitmex": 
      new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_bitmex(symbol, kline_size, data_df=None, save = False):
    if data_df is None: 
      data_df = pd.DataFrame()

    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    print('All caught up..!')
    return data_df

In [ ]:
# Load the data, keep only relevant columns and filter by date

data=get_all_bitmex('XBTUSD', '5m', save=True)
data = data.reset_index(drop=False)[['timestamp', 'open','high','low','close','volume']]
start_date = '2017-09-01'
data = data[data['timestamp'] >= start_date].reset_index(drop=True)

In [ ]:
# Transform 5m frequency to 15m

new_data = pd.DataFrame([], columns=data.columns)

for i in range(2, len(data), 3):
  a = np.zeros(6)
  a[1] = data.loc[i-2].open
  a[2] = data.loc[i-2:i].high.max()
  a[3] = data.loc[i-2:i].low.min()
  a[4] = data.loc[i].close
  a[5] = data.loc[i-2:i].volume.sum()
  new_data.loc[len(new_data)] = a
  new_data.loc[len(new_data)-1, 'timestamp'] = data.loc[i].timestamp

In [ ]:
# Save the final dataframe

new_data.to_csv(os.getcwd()+'/bitmex_data.csv')